<a href="https://colab.research.google.com/github/DanielaPortilloMercado/Inteligencia-Artificial/blob/main/PARCIALES/SEGUNDOPARCIAL/Pregunta2/SegundoParcialPUNTO2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SEGUNDO PARCIAL

Aprendizaje por refuerzo

DANIELA PORTILLO MERCADO
CARRERA: Ingenieria de Sistemas


Representa el tablero del juego

0 si está vacía

1 si la ocupa el jugador 1

-1 si la ocupa el jugador 2



# CUATRO EN RAYA
Con el objetivo de ilustrar la idea general del axr vamos a considerar un ejemplo en detalle: el juego del tres en raya. En este juego, dos jugadores se turnan para dibujar una X o una O en un tablero con 4x4 posiciones. El primer jugador en conseguir dibujar tres figuras en una línea horizontal, vertical o diagonal, gana. Nuestro objetivo es conseguir un agente que sea capaz de ganar siempre a este juego.



In [1]:
import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((4,4))

    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")

    def is_game_over(self):
        # Comprobar filas y columnas
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1
        # Comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if 4 in diag_sums:
            return 1
        if -4 in diag_sums:
            return -1
        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    def reset(self):
       self.state = np.zeros((4, 4))

CONTROLADOR DEL JUEGO

Se encarga de manejar el flujo de una partida entre dos agentes.
Inicializa dos jugadores asignandoles simbolos de 1  y -1
Ejecuta multiples partidad de autoaprendizaje por defecto 100


Al finalizar la partida:

Si hay empate, ambos jugadores reciben 0.5.

Si hay un ganador, recibe 1, y el perdedor 0.



In [2]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins


    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0: # empate
            for player in self.players:
                player.reward(0.5)
        else: # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

CLASE AGENT:
Define el comportamiento del agente que aprende:

value_function: tabla de valores para estados.

alpha: tasa de aprendizaje.

prob_exp: probabilidad de explorar nuevas jugadas.

positions: almacena los estados recorridos en una partida.

Recorre todas las posiciones jugadas al revés y actualiza sus valores según la recompensa final.

In [3]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.4):
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))


    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

ENTRENAMIENTO DE LOS AGENTES

Se crean dos agentes y se entrena al agente1 con exploracion usando 20000 partidas


In [4]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(20000)

100%|██████████| 20000/20000 [05:44<00:00, 58.13it/s]


[6225, 5580]

VISUALIZAR LA FUNCION DE VALOR
Se ordena la tabla de valores de estados aprendidos de mayor a menor, y se visualiza en forma de DataFrame.


In [5]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 0. 0. 0. 0. 0. 0. 0. -1. -1. -1. ...,1.000000
1,[ 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. -...,0.999969
2,[ 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. -...,0.999939
3,[ 0. 0. -1. 1. 0. 0. -1. 1. 0. 0. 0. ...,0.999939
4,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. 0. -1. ...,0.999512
...,...,...
148445,[ 1. 0. -1. -1. 0. 0. -1. -1. 1. -1. 1. ...,0.000000
148446,[ 0. 1. 1. 1. 0. 0. 1. 1. -1. -1. -1. -...,0.000000
148447,[ 0. 1. 1. 1. 0. 0. 1. 1. -1. -1. 0. -...,0.000000
148448,[ 0. 1. 1. -1. 0. 1. 1. -1. 0. 1. 0. -...,0.000000


GUARDAR AGENTE ENTRENADO

Se guarda el conocimiento del agente (su función de valor) en un archivo .pickle para su uso posterior.

In [7]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)